In [0]:
%run ./setup

## Load model

Retrieve model form 'Models'

From UI to copy the model name from : dbacademy.labuser9128531_1738705451.rf2

In [0]:
import mlflow
#                                      Stage/version
#                       Model name          |              output
#                           |               |                 |
mlflow.set_registry_uri('databricks-uc')
modelURL = "models:/" + 'dbacademy.labuser9128531_1738705451.rf2' + "@production"    #        |
print("Retrieving model " + modelURL)                #        |
predict_churn_udf = mlflow.pyfunc.spark_udf(spark, modelURL, "int")
#We can use the function in SQL
spark.udf.register("predict_churn", predict_churn_udf)